<a href="https://colab.research.google.com/github/New-Languages-for-NLP/yiddish/blob/main/research/notebooks/Spacy_Yiddish_Training_(Colab).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img width=50% src="https://github.com/New-Languages-for-NLP/course-materials/raw/main/w2/using-inception-data/newnlp_notebook.png" />

For full documentation on this project, see [here](https://new-languages-for-nlp.github.io/course-materials/w2/using-inception-data/New%20Language%20Training.html)
 

# 1 Prepare the Notebook Environment

In [1]:
%%capture
#@title The Colab runtime comes with spaCy v2 and needs to be upgraded to v3.
#@markdown This project uses the GPU by default, if you need to use just the CPU, just uncheck the box below.
GPU = True #@param {type:"boolean"}

# Install spaCy v3 and libraries for GPUs and transformers
!pip install spacy --upgrade
if GPU:
    !pip install 'spacy[transformers,cuda111]'
#!pip install wandb spacy-huggingface-hub

The notebook will pull project files from your GitHub repository.  

Note that you need to set the langugage (lang), treebank (same as the repo name), test_size and package name in the project.yml file in your repository.  

In [2]:
#@title Enter your language's repository name. 
#@markdown If the repo is private, please check the "private_repo" box and include an [access token](https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/creating-a-personal-access-token).
private_repo = False #@param {type:"boolean"}
repo_name = "yiddish" #@param {type:"string"}
branch = "main"


!rm -rf /content/newlang_project
!rm -rf $repo_name
if private_repo:
    git_access_token = "" #@param {type:"string"}
    git_url = f"https://{git_access_token}@github.com/New-Languages-for-NLP/{repo_name}/"
    !git clone $git_url  -b $branch
    !cp -r ./$repo_name/newlang_project .  
    !mkdir newlang_project/assets/
    !mkdir newlang_project/configs/
    #!mkdir newlang_project/corpus/
    !mkdir newlang_project/metrics/
    !mkdir newlang_project/packages/
    !mkdir newlang_project/training/
    !mkdir newlang_project/assets/$repo_name
    !cp -r ./$repo_name/* newlang_project/assets/$repo_name/
    !rm -rf ./$repo_name
else:
    !python -m spacy project clone newlang_project --repo https://github.com/New-Languages-for-NLP/$repo_name --branch $branch
    !python -m spacy project assets /content/newlang_project

✔ Cloned 'newlang_project' from New-Languages-for-NLP/yiddish
/content/newlang_project
✔ Your project is now ready!
To fetch the assets, run:
python -m spacy project assets /content/newlang_project
ℹ Fetching 1 asset(s)
✔ Downloaded asset /content/newlang_project/assets/yiddish


In [3]:
# Install the custom language object from Cadet 
!python -m spacy project run install /content/newlang_project


================================== install ==================================
Running command: rm -rf lang
Running command: mkdir lang
Running command: mkdir lang/yi
Running command: cp -r assets/yiddish/2_new_language_object/ lang/yi/yi
Running command: mv lang/yi/yi/setup.py lang/yi/
Running command: /usr/bin/python3 -m pip install -e lang/yi
Obtaining file:///content/newlang_project/lang/yi
  Running setup.py develop for yi


## [**Pretraining**](https://spacy.io/usage/embeddings-transformers#pretraining) (optional)

With pretraining, spaCy will learn initial token embeddings from your raw text files.  This can lead to significant improvement when you don't have a lot of data. 
*  
* It will load the raw text files (.txt) from your `0_original_texts` folder in GitHub

In [4]:
!python -m spacy project run prep-rawtext /content/newlang_project


✘ Can't find command or workflow 'prep-rawtext' in project.yml
Available commands: install, convert, split, debug, train, evaluate, package,
document. Available workflows: all



In [5]:
!python -m spacy pretrain /content/newlang_project/configs/config_pretrain.cfg ./pretrain --gpu-id 0

Usage: python -m spacy pretrain 
           [OPTIONS] CONFIG_PATH OUTPUT_DIR
Try 'python -m spacy pretrain --help' for help.

Error: Invalid value for 'CONFIG_PATH': File '/content/newlang_project/configs/config_pretrain.cfg' does not exist.


# 2 Prepare the Data for Training

In [6]:
#@title (optional) cell to correct a problem when your tokens have no pos value
%%writefile /usr/local/lib/python3.7/dist-packages/spacy/training/converters/conllu_to_docs.py
import re

from .conll_ner_to_docs import n_sents_info
from ...training import iob_to_biluo, biluo_tags_to_spans
from ...tokens import Doc, Token, Span
from ...vocab import Vocab
from wasabi import Printer


def conllu_to_docs(
    input_data,
    n_sents=10,
    append_morphology=False,
    ner_map=None,
    merge_subtokens=False,
    no_print=False,
    **_
):
    """
    Convert conllu files into JSON format for use with train cli.
    append_morphology parameter enables appending morphology to tags, which is
    useful for languages such as Spanish, where UD tags are not so rich.

    Extract NER tags if available and convert them so that they follow
    BILUO and the Wikipedia scheme
    """
    MISC_NER_PATTERN = "^((?:name|NE)=)?([BILU])-([A-Z_]+)|O$"
    msg = Printer(no_print=no_print)
    n_sents_info(msg, n_sents)
    sent_docs = read_conllx(
        input_data,
        append_morphology=append_morphology,
        ner_tag_pattern=MISC_NER_PATTERN,
        ner_map=ner_map,
        merge_subtokens=merge_subtokens,
    )
    sent_docs_to_merge = []
    for sent_doc in sent_docs:
        sent_docs_to_merge.append(sent_doc)
        if len(sent_docs_to_merge) % n_sents == 0:
            yield Doc.from_docs(sent_docs_to_merge)
            sent_docs_to_merge = []
    if sent_docs_to_merge:
        yield Doc.from_docs(sent_docs_to_merge)


def has_ner(input_data, ner_tag_pattern):
    """
    Check the MISC column for NER tags.
    """
    for sent in input_data.strip().split("\n\n"):
        lines = sent.strip().split("\n")
        if lines:
            while lines[0].startswith("#"):
                lines.pop(0)
            for line in lines:
                parts = line.split("\t")
                id_, word, lemma, pos, tag, morph, head, dep, _1, misc = parts
                for misc_part in misc.split("|"):
                    if re.match(ner_tag_pattern, misc_part):
                        return True
    return False


def read_conllx(
    input_data,
    append_morphology=False,
    merge_subtokens=False,
    ner_tag_pattern="",
    ner_map=None,
):
    """Yield docs, one for each sentence"""
    vocab = Vocab()  # need vocab to make a minimal Doc
    for sent in input_data.strip().split("\n\n"):
        lines = sent.strip().split("\n")
        if lines:
            while lines[0].startswith("#"):
                lines.pop(0)
            doc = conllu_sentence_to_doc(
                vocab,
                lines,
                ner_tag_pattern,
                merge_subtokens=merge_subtokens,
                append_morphology=append_morphology,
                ner_map=ner_map,
            )
            yield doc


def get_entities(lines, tag_pattern, ner_map=None):
    """Find entities in the MISC column according to the pattern and map to
    final entity type with `ner_map` if mapping present. Entity tag is 'O' if
    the pattern is not matched.

    lines (str): CONLL-U lines for one sentences
    tag_pattern (str): Regex pattern for entity tag
    ner_map (dict): Map old NER tag names to new ones, '' maps to O.
    RETURNS (list): List of BILUO entity tags
    """
    miscs = []
    for line in lines:
        parts = line.split("\t")
        id_, word, lemma, pos, tag, morph, head, dep, _1, misc = parts
        if "-" in id_ or "." in id_:
            continue
        miscs.append(misc)

    iob = []
    for misc in miscs:
        iob_tag = "O"
        for misc_part in misc.split("|"):
            tag_match = re.match(tag_pattern, misc_part)
            if tag_match:
                prefix = tag_match.group(2)
                suffix = tag_match.group(3)
                if prefix and suffix:
                    iob_tag = prefix + "-" + suffix
                    if ner_map:
                        suffix = ner_map.get(suffix, suffix)
                        if suffix == "":
                            iob_tag = "O"
                        else:
                            iob_tag = prefix + "-" + suffix
                break
        iob.append(iob_tag)
    return iob_to_biluo(iob)


def conllu_sentence_to_doc(
    vocab,
    lines,
    ner_tag_pattern,
    merge_subtokens=False,
    append_morphology=False,
    ner_map=None,
):
    """Create an Example from the lines for one CoNLL-U sentence, merging
    subtokens and appending morphology to tags if required.

    lines (str): The non-comment lines for a CoNLL-U sentence
    ner_tag_pattern (str): The regex pattern for matching NER in MISC col
    RETURNS (Example): An example containing the annotation
    """
    # create a Doc with each subtoken as its own token
    # if merging subtokens, each subtoken orth is the merged subtoken form
    if not Token.has_extension("merged_orth"):
        Token.set_extension("merged_orth", default="")
    if not Token.has_extension("merged_lemma"):
        Token.set_extension("merged_lemma", default="")
    if not Token.has_extension("merged_morph"):
        Token.set_extension("merged_morph", default="")
    if not Token.has_extension("merged_spaceafter"):
        Token.set_extension("merged_spaceafter", default="")
    words, spaces, tags, poses, morphs, lemmas = [], [], [], [], [], []
    heads, deps = [], []
    subtok_word = ""
    in_subtok = False
    for i in range(len(lines)):
        line = lines[i]
        parts = line.split("\t")
        id_, word, lemma, pos, tag, morph, head, dep, _1, misc = parts
        if "." in id_:
            continue
        if "-" in id_:
            in_subtok = True
        if "-" in id_:
            in_subtok = True
            subtok_word = word
            subtok_start, subtok_end = id_.split("-")
            subtok_spaceafter = "SpaceAfter=No" not in misc
            continue
        if merge_subtokens and in_subtok:
            words.append(subtok_word)
        else:
            words.append(word)
        if in_subtok:
            if id_ == subtok_end:
                spaces.append(subtok_spaceafter)
            else:
                spaces.append(False)
        elif "SpaceAfter=No" in misc:
            spaces.append(False)
        else:
            spaces.append(True)
        if in_subtok and id_ == subtok_end:
            subtok_word = ""
            in_subtok = False
        id_ = int(id_) - 1
        head = (int(head) - 1) if head not in ("0", "_") else id_
        tag = pos if tag == "_" else tag
        morph = morph if morph != "_" else ""
        dep = "ROOT" if dep == "root" else dep
        lemmas.append(lemma)
        if pos == "_":
            pos = ""
        poses.append(pos)
        tags.append(tag)
        morphs.append(morph)
        heads.append(head)
        deps.append(dep)

    doc = Doc(
        vocab,
        words=words,
        spaces=spaces,
        tags=tags,
        pos=poses,
        deps=deps,
        lemmas=lemmas,
        morphs=morphs,
        heads=heads,
    )
    for i in range(len(doc)):
        doc[i]._.merged_orth = words[i]
        doc[i]._.merged_morph = morphs[i]
        doc[i]._.merged_lemma = lemmas[i]
        doc[i]._.merged_spaceafter = spaces[i]
    ents = get_entities(lines, ner_tag_pattern, ner_map)
    doc.ents = biluo_tags_to_spans(doc, ents)

    if merge_subtokens:
        doc = merge_conllu_subtokens(lines, doc)

    # create final Doc from custom Doc annotation
    words, spaces, tags, morphs, lemmas, poses = [], [], [], [], [], []
    heads, deps = [], []
    for i, t in enumerate(doc):
        words.append(t._.merged_orth)
        lemmas.append(t._.merged_lemma)
        spaces.append(t._.merged_spaceafter)
        morphs.append(t._.merged_morph)
        if append_morphology and t._.merged_morph:
            tags.append(t.tag_ + "__" + t._.merged_morph)
        else:
            tags.append(t.tag_)
        poses.append(t.pos_)
        heads.append(t.head.i)
        deps.append(t.dep_)

    doc_x = Doc(
        vocab,
        words=words,
        spaces=spaces,
        tags=tags,
        morphs=morphs,
        lemmas=lemmas,
        pos=poses,
        deps=deps,
        heads=heads,
    )
    doc_x.ents = [Span(doc_x, ent.start, ent.end, label=ent.label) for ent in doc.ents]

    return doc_x


def merge_conllu_subtokens(lines, doc):
    # identify and process all subtoken spans to prepare attrs for merging
    subtok_spans = []
    for line in lines:
        parts = line.split("\t")
        id_, word, lemma, pos, tag, morph, head, dep, _1, misc = parts
        if "-" in id_:
            subtok_start, subtok_end = id_.split("-")
            subtok_span = doc[int(subtok_start) - 1 : int(subtok_end)]
            subtok_spans.append(subtok_span)
            # create merged tag, morph, and lemma values
            tags = []
            morphs = {}
            lemmas = []
            for token in subtok_span:
                tags.append(token.tag_)
                lemmas.append(token.lemma_)
                if token._.merged_morph:
                    for feature in token._.merged_morph.split("|"):
                        field, values = feature.split("=", 1)
                        if field not in morphs:
                            morphs[field] = set()
                        for value in values.split(","):
                            morphs[field].add(value)
            # create merged features for each morph field
            for field, values in morphs.items():
                morphs[field] = field + "=" + ",".join(sorted(values))
            # set the same attrs on all subtok tokens so that whatever head the
            # retokenizer chooses, the final attrs are available on that token
            for token in subtok_span:
                token._.merged_orth = token.orth_
                token._.merged_lemma = " ".join(lemmas)
                token.tag_ = "_".join(tags)
                token._.merged_morph = "|".join(sorted(morphs.values()))
                token._.merged_spaceafter = (
                    True if subtok_span[-1].whitespace_ else False
                )

    with doc.retokenize() as retokenizer:
        for span in subtok_spans:
            retokenizer.merge(span)

    return doc

Overwriting /usr/local/lib/python3.7/dist-packages/spacy/training/converters/conllu_to_docs.py


In [7]:
# Convert the conllu files from inception to spaCy binary format
# Read the conll files with ner data and as ents to spaCy docs 
!python -m spacy project run convert /content/newlang_project


================================== convert ==================================
Running command: /usr/bin/python3 scripts/convert.py assets/yiddish/3_inception_export 10 yi
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1 documents):
corpus/conllu/Forverts_312_allPs.spacy
ℹ Grouping every 10 sentences into a document.
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.7/dist-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/usr/local/lib/python3.7/dist-packages/spacy/cli/_util.py", line 71, in setup_cli
    command(prog_name=COMMAND)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/usr/local/lib/python3.7/dist-packages/click/core.py", line 782, in main
    rv =

In [8]:
# test/train split 
!python -m spacy project run split /content/newlang_project 


=================================== split ===================================
Running command: /usr/bin/python3 scripts/split.py 0.2 11 yi
🚂 Created 28 training docs
😊 Created 6 validation docs
🧪  Created 2 test docs


In [9]:
# Debug the data
!python -m spacy project run debug /content/newlang_project 


=================================== debug ===================================
Running command: /usr/bin/python3 -m spacy debug data configs/config.cfg

============================ Data file validation ============================
✔ Pipeline can be initialized with data
✔ Corpus is loadable

=============================== Training stats ===============================
Language: yi
Training pipeline: tok2vec, tagger, parser, ner
28 training docs
6 evaluation docs
✔ No overlap between training and evaluation data
✘ Low number of examples to train a new pipeline (28)

============================== Vocab & Vectors ==============================
ℹ 7447 total word(s) in the data (2103 unique)
ℹ No word vectors present in the package

========================== Named Entity Recognition ==========================
ℹ 6 label(s)
0 missing value(s) (tokens with '-' label)
⚠ Low number of examples for label 'ORG' (10)
⚠ Low number of examples for label 'm/watch\' (3)
⚠ Low number of examples for

# 3 Model Training 

If your project file uses Weights and Biases to monitor model training, you'll need to create an account at [wandb.ai](https://wandb.ai/site) and get an API key.  

In [10]:
# train the model
!python -m spacy project run train /content/newlang_project 


=================================== train ===================================
Running command: /usr/bin/python3 -m spacy train configs/config.cfg --output training/yiddish --gpu-id 0 --nlp.lang=yi
✔ Created output directory: training/yiddish
ℹ Saving to output directory: training/yiddish
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-05-12 15:15:00,808] [INFO] Set up nlp object from config
[2022-05-12 15:15:00,822] [INFO] Pipeline: ['tok2vec', 'tagger', 'parser', 'ner']
[2022-05-12 15:15:00,828] [INFO] Created vocabulary
[2022-05-12 15:15:00,829] [INFO] Finished initializing nlp object
[2022-05-12 15:16:13,000] [INFO] Initialized pipeline components: ['tok2vec', 'tagger', 'parser', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'tagger', 'parser', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS TAGGER  LOSS PARSER  LOSS NER  TAG_

If you get `ValueError: Could not find gold transition - see logs above.`  
You may not have sufficent data to train on: https://github.com/explosion/spaCy/discussions/7282

In [11]:
# Evaluate the model using the test data
!python -m spacy project run evaluate /content/newlang_project 


================================== evaluate ==================================
Running command: /usr/bin/python3 -m spacy evaluate ./training/yiddish/model-best ./corpus/converted/test.spacy --output ./metrics/yiddish.json --gpu-id 0
ℹ Using GPU: 0

================================== Results ==================================

TOK      100.00
TAG      52.88 
UAS      100.00
LAS      0.00  
NER P    0.00  
NER R    0.00  
NER F    0.00  
SENT P   100.00
SENT R   100.00
SENT F   100.00
SPEED    1878  


=============================== LAS (per type) ===============================

          P      R      F
_      0.00   0.00   0.00
root   0.00   0.00   0.00


=============================== NER (per type) ===============================

         P      R      F
PER   0.00   0.00   0.00
LOC   0.00   0.00   0.00
ORG   0.00   0.00   0.00

✔ Saved results to metrics/yiddish.json


In [12]:
# Find the path for your meta.json file
# You'll need to add newlang_project/ +  the path from the training step just after "✔ Saved pipeline to output directory"
!ls /content/newlang_project/training/yiddish/model-last

config.cfg  meta.json  ner  parser  tagger  tok2vec  tokenizer	vocab


In [13]:
#Update meta.json
import spacy 
import srsly 

# Change path to match that from the training cell where it says "✔ Saved pipeline to output directory"
meta_path = "/content/newlang_project/training/yiddish/model-last/meta.json"

# Replace values below for your project
my_meta = { 
    "lang":"yi",
    "name":"yiddish_sm",
    "version":"0.0.1",
    "description":"Yiddish pipeline optimized for CPU. Components: tok2vec, tagger, parser, senter, lemmatizer.",
    "author":"New Languages for NLP",
    "email":"newnlp@princeton.edu",
    "url":"https://newnlp.princeton.edu",
    "license":"MIT", 
    }
meta = spacy.util.load_meta(meta_path)
meta.update(my_meta)
srsly.write_json(meta_path, meta)

### Download the trained model to your computer.


In [14]:
# Save the model to disk in a format that can be easily  downloaded and re-used.
!python -m spacy package /content/newlang_project/training/yiddish/model-last newlang_project/export 

ℹ Building package artifacts: sdist
✔ Loaded meta.json from file
/content/newlang_project/training/yiddish/model-last/meta.json
✔ Generated README.md from meta.json
✔ Successfully created package directory 'yi_yiddish_sm-0.0.1'
newlang_project/export/yi_yiddish_sm-0.0.1
running sdist
running egg_info
creating yi_yiddish_sm.egg-info
writing yi_yiddish_sm.egg-info/PKG-INFO
writing dependency_links to yi_yiddish_sm.egg-info/dependency_links.txt
writing entry points to yi_yiddish_sm.egg-info/entry_points.txt
writing requirements to yi_yiddish_sm.egg-info/requires.txt
writing top-level names to yi_yiddish_sm.egg-info/top_level.txt
writing manifest file 'yi_yiddish_sm.egg-info/SOURCES.txt'
reading manifest file 'yi_yiddish_sm.egg-info/SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'yi_yiddish_sm.egg-info/SOURCES.txt'
running check
creating yi_yiddish_sm-0.0.1
creating yi_yiddish_sm-0.0.1/yi_yiddish_sm
creating yi_yiddish_sm-0.0.1/yi_yiddish_sm.egg-info
creating yi

In [18]:
from google.colab import files
# replace with the path in the previous cell under "✔ Successfully created zipped Python package"
files.download('newlang_project/export/yi_yiddish_sm-0.0.1/dist/yi_yiddish_sm-0.0.1.tar.gz')

# once on your computer, you can pip install yi_yiddish_sm-0.0.1.tar.gz
# Be sure to add the file to the 4_trained_models folder in GitHub

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Using your trained model 

In [1]:
# Install the model as a module 
!pip install newlang_project/export/yi_yiddish_sm-0.0.1/dist/yi_yiddish_sm-0.0.1.tar.gz
# If you're using Jupyter or Colab, you may need to restart the runtime after installation of your model for it to be available. 


Processing ./newlang_project/export/yi_yiddish_sm-0.0.1/dist/yi_yiddish_sm-0.0.1.tar.gz
DEPRECATION: Source distribution is being reinstalled despite an installed package having the same name and version as the installed package. pip 21.2 will remove support for this functionality. A possible replacement is use --force-reinstall. You can find discussion regarding this at https://github.com/pypa/pip/issues/8711.
  Created wheel for yi-yiddish-sm: filename=yi_yiddish_sm-0.0.1-py3-none-any.whl size=7715422 sha256=f108b2bdd9ad93982be3819ddbdb6137b800a1d2053a5f67fa5f9c59c1a9e855
  Stored in directory: /root/.cache/pip/wheels/73/bc/62/9998d02ad7ee2fbf9ddb7eaeea603e1b3d81ddcb68e635cb90
Successfully built yi-yiddish-sm
  Attempting uninstall: yi-yiddish-sm
    Found existing installation: yi-yiddish-sm 0.0.1
    Uninstalling yi-yiddish-sm-0.0.1:
      Successfully uninstalled yi-yiddish-sm-0.0.1


In [2]:
#To import the model, use the language code (in this case 'yi' followed by the name ("yiddish_sm")) 
# You don't need the version information: '-0.0.1'
#So ✔ Successfully created package 'yi_yiddish_sm-0.0.1'
# Becomes yi_yiddish_sm 
import spacy 
nlp = spacy.load('yi_yiddish_sm')
doc = nlp("איך האב א היים אין ישראל")
for token in doc:
  print(token.text,token.pos_)
for ent in doc.ents:
  print(ent)

איך 
האב 
א 
היים 
אין 
ישראל 


In [3]:
doc = nlp("איך האב א היים אין תל-אביב")
for token in doc:
  print(token.text,token.pos_)
for ent in doc.ents:
  print(ent)

איך 
האב 
א 
היים 
אין 
תל 
- 
אביב 


In [4]:
doc = nlp("איך וווין נישט אין תל־אביב")
for token in doc:
  print(token.text,token.pos_)
for ent in doc.ents:
  print(ent)

איך 
וווין 
נישט 
אין 
תל־אביב 
